In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import random
tf.reset_default_graph()

In [2]:
# Data Preprocessing and creating dictionary  

training_data = open('dinos.txt','r').read().lower()
chars = list(set(training_data))
data_size,vocab_size = len(training_data),len(chars)
print("The size of total Chars is %d and size of unique is %d " %(data_size,vocab_size))
char_to_ix = {ch:i for i,ch in enumerate(sorted(chars))}
ix_to_char = {i:ch for i,ch in enumerate(sorted(chars))}

The size of total Chars is 19909 and size of unique is 27 


In [3]:
# Storing the log for tensorboard
logs_path = '/Users/davidjoy/Desktop/log'
writer = tf.summary.FileWriter(logs_path)

In [4]:
# no of hidden cells or units in rnn
n_hidden = 128

# hyperparameters 

learning_rate = .001
training_iters = 200000
display_step = 5000
time_steps = 3

In [5]:
x = tf.placeholder("float",[None,time_steps,1])
Y = tf.placeholder("float",[None,vocab_size])

weights = {'out':tf.Variable(tf.random_normal([n_hidden,vocab_size]))}
biases = {'out':tf.Variable(tf.random_normal([vocab_size]))}

In [6]:
# Defining the RNN block 

def RNN(x,weights,biases):
    x = tf.reshape(x,[-1,time_steps])
    x = tf.split(x,time_steps,1)
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden),rnn.BasicLSTMCell(n_hidden)])
    outputs,states = rnn.static_rnn(rnn_cell,x,dtype=tf.float32)
    return tf.matmul(outputs[-1],weights['out'])+biases['out']

In [7]:
# Predictor 

pred = RNN(x,weights,biases)

In [8]:
# loss and optimization 
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,time_steps+1)
    end_offset = time_steps + 1
    acc_total = 0
    loss_total = 0
    
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, time_steps+1)

        symbols_in_keys = [ [char_to_ix[ str(training_data[i])]] for i in range(offset, offset+time_steps) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, time_steps, 1])

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[char_to_ix[str(training_data[offset+time_steps])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, Y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + time_steps)]
            symbols_out = training_data[offset + time_steps]
            symbols_out_pred = ix_to_char[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (time_steps+1)
    print("Optimization Finished!")
    print("Run on command line.")
    print("\ttensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % time_steps
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != time_steps:
            continue
        try:
            symbols_in_keys = [char_to_ix[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, time_steps, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s%s" % (sentence,ix_to_char[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Iter= 5000, Average Loss= 2.599234, Average Accuracy= 22.94%
['p', 'h', 'o'] - [l] vs [a]
Iter= 10000, Average Loss= 2.340719, Average Accuracy= 32.78%
['l', 'u', 's'] - [
] vs [
]
Iter= 15000, Average Loss= 2.207374, Average Accuracy= 36.28%
['\n', 'a', 'e'] - [p] vs [o]
Iter= 20000, Average Loss= 2.210960, Average Accuracy= 36.60%
['i', 'l', 'i'] - [s] vs [o]
Iter= 25000, Average Loss= 2.174942, Average Accuracy= 37.22%
['g', 'e', 'n'] - [i] vs [o]
Iter= 30000, Average Loss= 2.117689, Average Accuracy= 38.44%
['l', 'b', 'e'] - [r] vs [r]
Iter= 35000, Average Loss= 2.075203, Average Accuracy= 39.78%
['e', 't', 'o'] - [p] vs [s]
Iter= 40000, Average Loss= 2.032519, Average Accuracy= 40.48%
['n', 'a', 'x'] - [
] vs [o]
Iter= 45000, Average Loss= 2.047342, Average Accuracy= 41.64%
['m', 'o', 's'] - [a] vs [a]
Iter= 50000, Average Loss= 2.007288, Average Accuracy= 41.60%
['u', 'r', 'u'] - [s] vs [s]
Iter= 55000, Average Loss= 1.950304, Average Accuracy= 43.68%
['s', 'a', 'u'] - [r] vs [r]